# 学習と検証
p2.xlargeで12時間


## 目標
1. PSPNetの学習と検証の実装
2. セマンティックセグメンテーションのファインチューニングを理解

## Library

In [1]:
# パッケージのimport
import random
import math
import time
import pandas as pd
import numpy as np

import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# 初期設定
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## DataLoader作成

In [3]:
from utils.dataloader import make_datapath_list, DataTransform, VOCDataset

# ファイルパスリスト作成
# ２章で使ったディレクトリにアクセスする
rootpath = "../2_objectdetection/data/VOCdevkit/VOC2012/"
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath=rootpath)

# Dataset作成
# (RGB)の色の平均値と標準偏差
color_mean = (0.485, 0.456, 0.406)
color_std = (0.229, 0.224, 0.225)

train_dataset = VOCDataset(train_img_list, train_anno_list, phase="train", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

val_dataset = VOCDataset(val_img_list, val_anno_list, phase="val", transform=DataTransform(
    input_size=475, color_mean=color_mean, color_std=color_std))

# DataLoader作成
batch_size = 4

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}


## ネットワークモデル作成

In [4]:
from utils.pspnet import PSPNet

# ADE20Kでは１５０クラス分類
# モデルの外側を作ってADE20Kの重みをダウンロードして最後のclassificatonの層を付け替える
# 付け替えた層の重みをxavierの初期値を使って初期化  今回はクラス分類なのでReLUではなくシグモイドを活性化関数に使うから
# ファインチューニングする

net = PSPNet(n_classes=150)
state_dict = torch.load('./weights/pspnet50_ADE20K.pth')   # aws上にダウンロードしてあれば良い

n_classes = 21
net.decode_feature.classification = nn.Conv2d(in_channels=512, out_channels=n_classes, kernel_size=1, stride=1, padding=0)
net.aux.classification = nn.Conv2d(in_channels=256, out_channels=n_classes, kernel_size=1, stride=1, padding=0)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:  # バイアス項がある場合
            nn.init.constant_(m.bias, 0.0)
            
net.decode_feature.classification.apply(weights_init)
net.aux.classification.apply(weights_init)

print('ネットワーク設定完了：学習済みモデルの重みをロードしました') 

    

ネットワーク設定完了：学習済みモデルの重みをロードしました


In [5]:
net

PSPNet(
  (feature_conv): FeatureMap_convolution(
    (cbnr_1): conv2DBatchNormRelu(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_2): conv2DBatchNormRelu(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (cbnr_3): conv2DBatchNormRelu(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (feature_res_1): ResidualBlockPSP(
    (block1): bottleNec

## 損失関数定義

In [6]:
class PSPLoss(nn.Module):
    """
    PSPNetの損失関数クラス
    """
    
    def __init__(self, aux_weight=0.4):
        super(PSPLoss, self).__init__()
        self.aux_weight = aux_weight
        
    def forward(self, outputs, targets):
        """
        損失関数の計算
        
        Parameters
        ----------------
        outputs : PSPNetの出力(tuple)
            (output=torch.Size([num_batch, 21, 475, 475]), output_aux=torch.Size([num_batch, 21, 475, 475]))。
        
        targets : [num_batch, 475, 475]
            正解のアノテーション情報
            
        Returns
        ----------------
        loss : テンソル
            損失の値（普通のlossとauxのlossを足したもの）
            
        """
        
        loss = F.cross_entropy(outputs[0], targets, reduction='mean')
        aux_loss = F.cross_entropy(outputs[1], targets, reduction='mean')
        
        return loss + self.aux_weight*aux_loss
    
criterion = PSPLoss(aux_weight=0.4)
        
        

## 最適化手法定義

In [7]:
# ファインチューニングなので学習率は小さめに設定しておく
optimizer = optim.SGD([
    {'params': net.feature_conv.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_res_2.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_1.parameters(), 'lr': 1e-3},
    {'params': net.feature_dilated_res_2.parameters(), 'lr': 1e-3},
    {'params': net.pyramid_pooling.parameters(), 'lr': 1e-3},
    {'params': net.decode_feature.parameters(), 'lr': 1e-2},
    {'params': net.aux.parameters(), 'lr': 1e-2},
], momentum=0.9, weight_decay=0.0001)

# スケジューラの設定
 # 今回はepochごとに学習率を小さくしていく
def lambda_epoch(epoch):
    max_epoch=30
    return math.pow((1-epoch/max_epoch), 0.9)

scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_epoch)

## 学習・検証

In [12]:
# モデルを学習させる関数
def train_model(net, detaloaders_dict, criterion, scheduler, optimizer, num_epoch):
    
    # gpu or cpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # 高速化させる
    torch.backends.cudnn.benchmark = True
    
    # 画像の枚数
    num_train_imgs = len(dataloaders_dict['train'].dataset)
    num_val_imgs = len(dataloaders_dict['val'].dataset)
    batch_size = dataloaders_dict['train'].batch_size
    
    iteration = 1
    logs = []
    batch_multiplier = 3
    
    # epochのループ
    for epoch in range(num_epochs):
        
        # 開始時刻の保存
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0
        epoch_val_loss = 0.0
        
        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')
        
        # trainとvalのループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
                scheduler.step()  # 最適化schedulerの更新
                optimizer.zero_grad()
                print('(train)')
                
            else:
                # 検証は５回に１回だけ行う
                if ((epoch+1)%5 == 0):
                    net.eval()
                    print('-------------')
                    print('(val)')
                else:
                    continue
            
            # minibatchを取り出すループ
            count = 0
            for images, anno_class_images, in dataloaders_dict[phase]:
                if images.size()[0] == 1:
                    continue
                    
                # gpuにデータを送る
                images = images.to(device)
                anno_class_images = anno_class_images.to(device)
                
                # multiple minibatchでのパラメータ更新
                if (phase == 'train') and (count==0):
                    optimizer.step()
                    optimizer.zero_grad()
                    count = batch_multiplier  # 3
                    
                # forwardの計算
                with torch.set_grad_enabled(phase=='train'):
                    outputs = net(images)
                    loss = criterion(outputs, anno_class_images.long()) / batch_multiplier
                    
                    # 訓練時はbackprop
                    if phase == 'train':
                        loss.backward()
                        count -= 1
                        
                        if iteration%10 == 0:
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('イテレーション{} || Loss: {:.4f} || 10iter: {:.4f} sec'.format(iteration, loss.item()/batch_size*batch_multiplier, duration))
                            t_iter_start = time.time()
                            
                        epoch_train_loss += loss.item()*batch_multiplier
                        iteration += 1
                        
                    # 検証時はloss加算するだけ
                    epoch_val_loss += loss.item() * batch_multiplier
                    
        # epochのphaseごとのlossと時間
        t_epoch_finish = time.time()
        print('------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} || Epoch_VAL_Loss: {:.4f}'.format(epoch+1, epoch_train_loss/num_train_imgs, epoch_val_loss/num_val_imgs))
        print('timer: {:.4f} sec'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()
        
        # ログの保存
        log_epoch = {'epoch': epoch+1, 
                     'train_loss': epoch_train_loss/num_train_imgs,
                    'val_loss': epoch_val_loss/num_val_imgs}
        logs.append(log_epoch)
        df = pd.DataFrame(logs)
        df.to_csv('log_output.csv')
        
    # 最後のネットワークを保存する
    torch.save(net.state_dict(), 'weights/pspnet50_'+str(epoch+1)+'.pth')

In [13]:
# 学習・検証の実施
# batch_size 8 だと　cuda out of memoryになってしまったので 4 にしてみた
num_epochs = 30
train_model(net, dataloaders_dict, criterion, scheduler, optimizer, num_epoch=num_epochs)

使用デバイス： cuda:0
-------------
Epoch 1/30
-------------
(train)
イテレーション10 || Loss: 0.5707 || 10iter: 26.0309 sec
イテレーション20 || Loss: 0.5270 || 10iter: 26.0447 sec
イテレーション30 || Loss: 0.5000 || 10iter: 26.0801 sec
イテレーション40 || Loss: 0.4331 || 10iter: 26.0617 sec
イテレーション50 || Loss: 0.3069 || 10iter: 26.0655 sec
イテレーション60 || Loss: 0.5151 || 10iter: 26.0040 sec
イテレーション70 || Loss: 0.4980 || 10iter: 26.0672 sec
イテレーション80 || Loss: 0.2151 || 10iter: 26.0345 sec
イテレーション90 || Loss: 0.3101 || 10iter: 26.0964 sec
イテレーション100 || Loss: 0.5349 || 10iter: 26.0574 sec
イテレーション110 || Loss: 0.3302 || 10iter: 26.0324 sec
イテレーション120 || Loss: 0.3808 || 10iter: 26.1123 sec
イテレーション130 || Loss: 0.2976 || 10iter: 26.0656 sec
イテレーション140 || Loss: 0.4505 || 10iter: 26.1194 sec
イテレーション150 || Loss: 0.2103 || 10iter: 26.1251 sec
イテレーション160 || Loss: 0.3192 || 10iter: 26.2047 sec
イテレーション170 || Loss: 0.4700 || 10iter: 26.1627 sec
イテレーション180 || Loss: 0.2571 || 10iter: 26.1640 sec
イテレーション190 || Loss: 0.2808 || 10iter: 26.0886 s

イテレーション1520 || Loss: 0.2571 || 10iter: 26.0602 sec
イテレーション1530 || Loss: 0.2218 || 10iter: 26.0685 sec
イテレーション1540 || Loss: 0.3203 || 10iter: 26.0490 sec
イテレーション1550 || Loss: 0.4831 || 10iter: 26.0845 sec
イテレーション1560 || Loss: 0.4073 || 10iter: 26.0239 sec
イテレーション1570 || Loss: 0.2026 || 10iter: 26.0859 sec
イテレーション1580 || Loss: 0.5153 || 10iter: 26.0438 sec
イテレーション1590 || Loss: 0.2155 || 10iter: 26.1078 sec
イテレーション1600 || Loss: 0.3649 || 10iter: 26.0865 sec
イテレーション1610 || Loss: 0.4950 || 10iter: 26.0903 sec
イテレーション1620 || Loss: 0.9208 || 10iter: 26.1016 sec
イテレーション1630 || Loss: 0.2263 || 10iter: 26.1272 sec
イテレーション1640 || Loss: 0.4235 || 10iter: 26.1845 sec
イテレーション1650 || Loss: 0.3600 || 10iter: 26.1603 sec
イテレーション1660 || Loss: 0.3208 || 10iter: 26.1414 sec
イテレーション1670 || Loss: 0.7818 || 10iter: 26.1273 sec
イテレーション1680 || Loss: 0.1860 || 10iter: 26.0987 sec
イテレーション1690 || Loss: 0.3069 || 10iter: 26.0989 sec
イテレーション1700 || Loss: 0.3534 || 10iter: 26.0660 sec
イテレーション1710 || Loss: 0.3849 || 

イテレーション3020 || Loss: 0.2994 || 10iter: 26.2445 sec
イテレーション3030 || Loss: 0.4164 || 10iter: 26.1399 sec
イテレーション3040 || Loss: 0.3660 || 10iter: 26.1302 sec
イテレーション3050 || Loss: 0.1568 || 10iter: 26.1072 sec
イテレーション3060 || Loss: 0.5461 || 10iter: 26.1340 sec
イテレーション3070 || Loss: 0.3479 || 10iter: 26.0901 sec
イテレーション3080 || Loss: 0.2710 || 10iter: 26.1083 sec
イテレーション3090 || Loss: 0.3096 || 10iter: 26.0954 sec
イテレーション3100 || Loss: 0.2932 || 10iter: 26.0594 sec
イテレーション3110 || Loss: 0.4932 || 10iter: 26.0693 sec
イテレーション3120 || Loss: 0.1875 || 10iter: 26.1299 sec
イテレーション3130 || Loss: 0.2573 || 10iter: 26.1399 sec
イテレーション3140 || Loss: 0.1554 || 10iter: 26.0900 sec
イテレーション3150 || Loss: 0.7201 || 10iter: 26.1158 sec
イテレーション3160 || Loss: 0.2337 || 10iter: 26.0539 sec
イテレーション3170 || Loss: 0.2181 || 10iter: 26.1540 sec
イテレーション3180 || Loss: 0.3635 || 10iter: 26.2114 sec
イテレーション3190 || Loss: 0.2121 || 10iter: 26.2020 sec
イテレーション3200 || Loss: 0.3746 || 10iter: 26.1478 sec
イテレーション3210 || Loss: 0.3889 || 

イテレーション4510 || Loss: 0.3168 || 10iter: 26.1592 sec
イテレーション4520 || Loss: 0.4088 || 10iter: 26.1714 sec
イテレーション4530 || Loss: 0.3581 || 10iter: 26.1854 sec
イテレーション4540 || Loss: 0.2786 || 10iter: 26.2058 sec
イテレーション4550 || Loss: 0.1851 || 10iter: 26.2334 sec
イテレーション4560 || Loss: 0.4967 || 10iter: 26.1234 sec
イテレーション4570 || Loss: 0.2589 || 10iter: 26.2148 sec
イテレーション4580 || Loss: 0.2863 || 10iter: 26.2266 sec
イテレーション4590 || Loss: 0.2343 || 10iter: 26.1086 sec
イテレーション4600 || Loss: 0.2887 || 10iter: 26.1055 sec
イテレーション4610 || Loss: 0.2711 || 10iter: 26.2016 sec
イテレーション4620 || Loss: 0.4723 || 10iter: 26.1551 sec
イテレーション4630 || Loss: 0.5936 || 10iter: 26.2270 sec
イテレーション4640 || Loss: 0.2254 || 10iter: 26.2866 sec
イテレーション4650 || Loss: 0.3610 || 10iter: 26.2277 sec
イテレーション4660 || Loss: 0.3458 || 10iter: 26.2209 sec
イテレーション4670 || Loss: 0.3119 || 10iter: 26.0648 sec
イテレーション4680 || Loss: 0.3804 || 10iter: 26.0926 sec
イテレーション4690 || Loss: 0.1816 || 10iter: 26.1242 sec
イテレーション4700 || Loss: 0.3478 || 

イテレーション6000 || Loss: 0.5357 || 10iter: 26.2122 sec
イテレーション6010 || Loss: 0.4339 || 10iter: 26.2124 sec
イテレーション6020 || Loss: 0.5601 || 10iter: 26.2641 sec
イテレーション6030 || Loss: 0.3964 || 10iter: 26.3767 sec
イテレーション6040 || Loss: 0.1470 || 10iter: 26.3005 sec
イテレーション6050 || Loss: 0.2678 || 10iter: 26.3473 sec
イテレーション6060 || Loss: 0.4901 || 10iter: 26.2560 sec
イテレーション6070 || Loss: 0.2146 || 10iter: 26.2719 sec
イテレーション6080 || Loss: 0.2398 || 10iter: 26.2415 sec
イテレーション6090 || Loss: 0.1137 || 10iter: 26.2103 sec
イテレーション6100 || Loss: 0.2454 || 10iter: 26.2710 sec
イテレーション6110 || Loss: 0.6101 || 10iter: 26.1941 sec
イテレーション6120 || Loss: 0.2380 || 10iter: 26.1912 sec
イテレーション6130 || Loss: 0.2492 || 10iter: 26.2572 sec
イテレーション6140 || Loss: 0.2386 || 10iter: 26.2363 sec
イテレーション6150 || Loss: 0.2901 || 10iter: 26.2240 sec
イテレーション6160 || Loss: 0.2001 || 10iter: 26.2795 sec
イテレーション6170 || Loss: 0.1823 || 10iter: 26.2404 sec
イテレーション6180 || Loss: 0.3208 || 10iter: 26.2381 sec
イテレーション6190 || Loss: 0.2490 || 

イテレーション7490 || Loss: 0.3176 || 10iter: 26.2666 sec
イテレーション7500 || Loss: 0.1408 || 10iter: 26.2849 sec
イテレーション7510 || Loss: 0.1922 || 10iter: 26.3461 sec
イテレーション7520 || Loss: 0.3006 || 10iter: 26.2498 sec
イテレーション7530 || Loss: 0.1393 || 10iter: 26.2987 sec
イテレーション7540 || Loss: 0.3310 || 10iter: 26.3661 sec
イテレーション7550 || Loss: 0.0943 || 10iter: 26.2889 sec
イテレーション7560 || Loss: 0.4140 || 10iter: 26.3381 sec
イテレーション7570 || Loss: 0.2076 || 10iter: 26.3610 sec
イテレーション7580 || Loss: 0.1983 || 10iter: 26.3454 sec
イテレーション7590 || Loss: 0.1204 || 10iter: 26.3734 sec
イテレーション7600 || Loss: 0.3913 || 10iter: 26.3750 sec
イテレーション7610 || Loss: 0.4252 || 10iter: 26.3759 sec
イテレーション7620 || Loss: 0.3901 || 10iter: 26.3351 sec
イテレーション7630 || Loss: 0.3187 || 10iter: 26.3760 sec
イテレーション7640 || Loss: 0.3378 || 10iter: 26.4442 sec
イテレーション7650 || Loss: 0.3254 || 10iter: 26.4197 sec
イテレーション7660 || Loss: 0.1207 || 10iter: 26.4084 sec
イテレーション7670 || Loss: 0.1762 || 10iter: 26.3332 sec
イテレーション7680 || Loss: 0.2615 || 

イテレーション8990 || Loss: 0.2323 || 10iter: 26.3093 sec
イテレーション9000 || Loss: 0.2730 || 10iter: 26.3951 sec
イテレーション9010 || Loss: 0.3219 || 10iter: 26.3560 sec
イテレーション9020 || Loss: 0.2773 || 10iter: 26.3369 sec
イテレーション9030 || Loss: 0.2474 || 10iter: 26.3014 sec
イテレーション9040 || Loss: 0.3037 || 10iter: 26.2986 sec
イテレーション9050 || Loss: 0.2639 || 10iter: 26.3459 sec
イテレーション9060 || Loss: 0.1589 || 10iter: 26.3474 sec
イテレーション9070 || Loss: 0.1389 || 10iter: 26.4084 sec
イテレーション9080 || Loss: 0.1853 || 10iter: 26.3578 sec
イテレーション9090 || Loss: 0.3094 || 10iter: 26.2781 sec
イテレーション9100 || Loss: 0.2710 || 10iter: 26.3063 sec
イテレーション9110 || Loss: 0.3332 || 10iter: 26.3383 sec
イテレーション9120 || Loss: 0.1263 || 10iter: 26.3630 sec
イテレーション9130 || Loss: 0.5653 || 10iter: 26.3345 sec
イテレーション9140 || Loss: 0.1869 || 10iter: 26.3726 sec
イテレーション9150 || Loss: 0.3300 || 10iter: 26.2849 sec
-------------
(val)
------------
epoch 25 || Epoch_TRAIN_Loss:0.2898 || Epoch_VAL_Loss: 0.6583
timer: 1456.7053 sec
-------------
Epo

UnboundLocalError: local variable 'values' referenced before assignment